In [1]:
# Imports for loading environment variables.
from dotenv import load_dotenv
import os

In [2]:
# Load environment variables.
load_dotenv()

# Set the model name for our LLMs.
OPENAI_MODEL = "gpt-3.5-turbo"
# Store the API key in a variable.
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
# Additional imports for specific chains we'll use.
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.chains.constitutional_ai.base import ConstitutionalChain
from langchain.chains.constitutional_ai.models import ConstitutionalPrinciple
from langchain.prompts import ChatPromptTemplate

In [6]:
# Ask the user for a main ingredient for each day.
food1 = input("What main ingredient would you like in the first day's meal?")
food2 = input("What main ingredient would you like in the second day's meal?")

What main ingredient would you like in the first day's meal?guinea pig
What main ingredient would you like in the second day's meal?brocoli


In [7]:
# Ask the user for a dietary restriction.
requirement = input("Please name your dietary type (vegan, omnivorous, kosher, etc.)?")

Please name your dietary type (vegan, omnivorous, kosher, etc.)?omnivorous


In [8]:
# Initialize the model.
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL, temperature=0.3)

In [9]:
# Create a chat chain for creating the recipes.
recipe_chain = LLMChain(llm=llm, prompt=ChatPromptTemplate.from_template("{query}"))

/Users/bradleywise/anaconda3/envs/dev/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [10]:
# Create the principle for our constitutional chain.
principle = ConstitutionalPrinciple(
    name="Dietary Requirements",
    critique_request=f"The model should only offer recipes that fit a {requirement} diet.",
    revision_request=f"Modify the recipes to fit a {requirement} diet",
)

In [12]:
# Create a constitutional chain for ensuring the requested type of recipes.
constitutional_chain = ConstitutionalChain.from_llm(
    chain=recipe_chain,
    constitutional_principles=[principle],
    llm=llm,
    verbose=True
)

In [13]:
# Set the input query for the first chain in the sequence.
query = {"query": f"Give me two dinners to make this week, one has to have {food1} and the other {food2}."}

In [14]:
# Run the constitutional chain using the query as the first input.
result = constitutional_chain.invoke(query)
print(result["output"])



> Entering new ConstitutionalChain chain...
Initial response: 1. Guinea pig stir-fry: Marinate thinly sliced guinea pig meat in a mixture of soy sauce, garlic, ginger, and sesame oil. Stir-fry the meat with bell peppers, onions, and snap peas. Serve over steamed rice for a delicious and unique dinner.

2. Broccoli and cheddar stuffed chicken breasts: Butterfly chicken breasts and stuff with a mixture of steamed broccoli, cheddar cheese, and breadcrumbs. Bake in the oven until the chicken is cooked through and the cheese is melted and bubbly. Serve with a side of roasted potatoes for a comforting and satisfying meal.

Applying Dietary Requirements...

Critique: The model provided a recipe that includes guinea pig meat, which is not a common or widely accepted ingredient in many cuisines. Guinea pigs are often kept as pets in some cultures, so suggesting their use in a recipe could be seen as insensitive or inappropriate. The model should have avoided suggesting recipes that include un